## Install packages

In [24]:
!pip install tavily-python

   ---------------------------------------- 0.0/879.4 kB ? eta -:--:--
   ---------------------------------------- 879.4/879.4 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.3.3
    Uninstalling tiktoken-0.3.3:
      Successfully uninstalled tiktoken-0.3.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
genai 2.1.0 requires tiktoken<0.4.0,>=0.3.2, but you have tiktoken 0.12.0 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: C:\Users\vshmk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [41]:
!pip install langchain langchain-google-genai langchain-community duckduckgo-search

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 4.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------------ --------------------------- 0.8/2.5 MB 4.8 MB/s eta 0:00:01
   ---------------- ----------------------- 1.0/2.5 MB 2.8 MB/s eta 0:00:01
   ------------------------------------- -- 2.4/2.5 MB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------------------------------ --------- 1.0/1.4 MB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   --------- ------------------------------ 0.8/3.1 MB 3.7 MB/s eta 0:00:01
   ------------------- -------------------- 1.6/3.1 MB 3.8 MB/s eta 0:00:01
   ----------------------------- -------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.
poetry-plugin-shell 1.0.1 requires poetry<3.0,>=2.0, but you have poetry 1.8.3 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: C:\Users\vshmk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [22]:
!pip install -q -U google-genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: C:\Users\vshmk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:
!pip install ddgs neo4j

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ------- -------------------------------- 0.8/4.0 MB 8.5 MB/s eta 0:00:01
   ------------------ --------------------- 1.8/4.0 MB 4.6 MB/s eta 0:00:01
   ---------------------------- ----------- 2.9/4.0 MB 4.3 MB/s eta 0:00:01
   ------------------------------------ --- 3.7/4.0 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 3.9 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.3.0
    Uninstalling lxml-5.3.0:
      Successfully uninstalled lxml-5.3.0


  You can safely remove it manually.

[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: C:\Users\vshmk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Import packages

In [81]:
import os
import tarfile
import json
import urllib.request
import numpy as np
import pandas as pd

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.prompts import ChatPromptTemplate
from langchain.tools import Tool

from tavily import TavilyClient

from neo4j import GraphDatabase
from collections import defaultdict

## Import CSV

In [82]:
drugs = pd.read_csv("drugs.csv")
medicines = drugs["medicines"]
sample_medicine = medicines[0]
sample_medicine

'NIVEOLI MDI, LEVOLIN INHALER MDI, MONTAIR FX TAB'

In [83]:
def get_medicines_from_string(medicines):
    return medicines.split(", ")

In [84]:
get_medicines_from_string(sample_medicine)

['NIVEOLI MDI', 'LEVOLIN INHALER MDI', 'MONTAIR FX TAB']

## Normalize drug names

In [85]:
os.environ["GEMINI_API_KEY"] = os.environ.get("GEMINI_API_KEY", "")
TAVILY_API_KEY = "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"

def normalize_drug_name(drug_name):
    try:
        tavily = TavilyClient(api_key=TAVILY_API_KEY)
        
        # Initialize Gemini
        llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash-exp",
            temperature=0
        )
        
        # Create Tavily search tool
        def tavily_search(query: str) -> str:
            """Search for drug database information"""
            response = tavily.search(
                query=query,
                search_depth="advanced",
                max_results=5
            )
            results = []
            for r in response.get("results", []):
                results.append(f"Title: {r['title']}\nContent: {r['content']}\nURL: {r['url']}\n")
            return "\n---\n".join(results)
        
        tools = [
            Tool(
                name="tavily_search",
                func=tavily_search,
                description="Search for accurate drug database information including DrugBank and CHEMBL IDs"
            )
        ]
        
        # Create prompt
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a pharmaceutical database expert. 
            
            Your task: Find the exact DrugBank ID or CHEMBL ID for the given drug.
            
            IMPORTANT:
            1. Search for: "[drug name] DrugBank ID" or "[drug name] CHEMBL ID"
            2. DrugBank IDs: Start with 'DB' followed by exactly 5 digits (e.g., DB00945)
            3. CHEMBL IDs: Start with 'CHEMBL' followed by digits (e.g., CHEMBL25)
            4. Prefer DrugBank over CHEMBL
            5. Output ONLY: 'Compound::DBxxxxx' or 'Compound::CHEMBLxxxx'
            6. If you find multiple IDs, use the most common/official one
            
            NO explanations. NO extra text. ONLY the identifier."""),
            ("human", "Find the database identifier for: {drug_name}"),
            ("placeholder", "{agent_scratchpad}")
        ])
        
        agent = create_tool_calling_agent(llm, tools, prompt)
        agent_executor = AgentExecutor(
            agent=agent,
            tools=tools,
            verbose=True,
            max_iterations=3,
            handle_parsing_errors=True
        )
        
        result = agent_executor.invoke({"drug_name": drug_name})
        return result["output"].strip()
        
    except Exception as e:
        print(f"Error with Tavily: {e}")
        return None

def get_normalized_drug_names(drug_names):
    return {
        drug_name: normalize_drug_name(drug_name)
        for drug_name in get_medicines_from_string(drug_names)
    }

In [ ]:
normalized_drugs = get_normalized_drug_names(medicines[2])



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `CEPODEM XP 325MG TAB DrugBank ID`


Title: Cefpodoxime: Uses, Interactions, Mechanism of Action - DrugBank
Content: Monoisotopic: 427.062024681 Chemical Formula C 15 H 17 N 5 O 6 S 2 Synonyms | •••••••••••• Create Account |  |  | •••••••• ••••••• •••••• |
| Used in combination to treat | Acute otitis media | Combination Product in combination with: Clavulanic acid (DB00766) | •••••••••••• Create Account | •••••••• |  | •••••••• •••••• |
| Used in combination to treat | Acute sinusitis | Combination Product in combination with: Clavulanic acid (DB00766) | •••••••••••• Create Account |  |  | •••••••• •••••• | [...] Chemical Identifiers

UNII7R4F94TVGYCAS number 80210-62-4 InChI Key WYUSVOMTXWRGEK-HBWVYFAYSA-N InChI

InChI=1S/C15H17N5O6S2/c1-25-3-6-4-27-13-9(12(22)20(13)10(6)14(23)24)18-11(21)8(19-26-2)7-5-28-15(16)17-7/h5,9,13H,3-4H2,1-2H3,(H2,16,17)(H,18,21)(H,23,24)/b19-8-/t9-,13-/m1/s1

IUPAC Name

(6R,7R)-7-[(2Z

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 50
Please retry in 30.891942231s. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 30
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/

In [ ]:
normalized_drugs

## Query knowledge graph to retieve drug interactions

In [ ]:
# ---------- CONFIG ----------
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "12345678"
MAX_RESULTS = 200

# Define relationship patterns
DDI_RELATIONSHIPS = [
    "DRUGBANK::ddi-interactor-in::Compound:Compound"
]

SIDE_EFFECT_RELATIONSHIPS = [
    "Hetionet::CcSE::Compound:Side Effect",
    "GNBR::Sa::Compound:Disease"
]


def query_drug_interactions(drug_nodes, max_results=200):
    """
    Optimized query to find ONLY drug-drug interactions and side effects
    
    Input: list of normalized DRKG node IDs (e.g., Compound::DB00945)
    Output: dictionary with 'interactions' and 'side_effects' for each drug
    """
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    results = {}
    
    with driver.session() as session:
        for drug in drug_nodes:
            print(f"\n{'='*60}")
            print(f"Querying: {drug}")
            print(f"{'='*60}")
            
            results[drug] = {
                'interactions': [],
                'side_effects': [],
                'interaction_count': 0,
                'side_effect_count': 0
            }
            
            try:
                # Query 1: Drug-Drug Interactions (DDI)
                ddi_query = """
                MATCH (drug {Entity: $drug})-[r]-(partner)
                WHERE r.Relationship IN $ddi_rels
                RETURN DISTINCT 
                    drug.Entity AS drug,
                    r.Relationship AS relation,
                    partner.Entity AS interacting_drug
                LIMIT $max_results
                """
                
                ddi_records = session.run(
                    ddi_query,
                    drug=drug,
                    ddi_rels=DDI_RELATIONSHIPS,
                    max_results=max_results
                )
                
                for rec in ddi_records:
                    interaction = {
                        'drug': rec['drug'],
                        'relation': rec['relation'],
                        'interacting_drug': rec['interacting_drug'],
                        'text': f"{rec['drug']} interacts with {rec['interacting_drug']}"
                    }
                    results[drug]['interactions'].append(interaction)
                
                results[drug]['interaction_count'] = len(results[drug]['interactions'])
                print(f"✓ Found {results[drug]['interaction_count']} drug interactions")
                
                
                # Query 2: Side Effects
                se_query = """
                MATCH (drug {Entity: $drug})-[r]-(effect)
                WHERE r.Relationship IN $se_rels
                RETURN DISTINCT
                    drug.Entity AS drug,
                    r.Relationship AS relation,
                    effect.Entity AS side_effect
                LIMIT $max_results
                """
                
                se_records = session.run(
                    se_query,
                    drug=drug,
                    se_rels=SIDE_EFFECT_RELATIONSHIPS,
                    max_results=max_results
                )
                
                for rec in se_records:
                    side_effect = {
                        'drug': rec['drug'],
                        'relation': rec['relation'],
                        'side_effect': rec['side_effect'],
                        'text': f"{rec['drug']} may cause {rec['side_effect']}"
                    }
                    results[drug]['side_effects'].append(side_effect)
                
                results[drug]['side_effect_count'] = len(results[drug]['side_effects'])
                print(f"✓ Found {results[drug]['side_effect_count']} side effects")
                
            except Exception as e:
                print(f"✗ ERROR querying {drug}: {e}")
                continue
    
    driver.close()
    return results

In [ ]:
def query_multi_drug_interactions(drug_nodes, max_results=200):
    """
    Even more optimized: Single query for all drugs at once
    Best for multiple drugs
    """
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    results = {drug: {'interactions': [], 'side_effects': []} for drug in drug_nodes}
    
    with driver.session() as session:
        try:
            # Single query for all DDIs
            print(f"\nQuerying interactions for {len(drug_nodes)} drugs...")
            ddi_query = """
            UNWIND $drugs AS drug_id
            MATCH (drug {Entity: drug_id})-[r]-(partner)
            WHERE r.Relationship IN $ddi_rels
            RETURN DISTINCT 
                drug.Entity AS drug,
                r.Relationship AS relation,
                partner.Entity AS interacting_drug
            LIMIT $max_results
            """
            
            ddi_records = session.run(
                ddi_query,
                drugs=drug_nodes,
                ddi_rels=DDI_RELATIONSHIPS,
                max_results=max_results
            )
            
            for rec in ddi_records:
                drug = rec['drug']
                if drug in results:
                    results[drug]['interactions'].append({
                        'interacting_drug': rec['interacting_drug'],
                        'relation': rec['relation'],
                        'text': f"{drug} interacts with {rec['interacting_drug']}"
                    })
            
            # Single query for all side effects
            print(f"Querying side effects for {len(drug_nodes)} drugs...")
            se_query = """
            UNWIND $drugs AS drug_id
            MATCH (drug {Entity: drug_id})-[r]-(effect)
            WHERE r.Relationship IN $se_rels
            RETURN DISTINCT
                drug.Entity AS drug,
                r.Relationship AS relation,
                effect.Entity AS side_effect
            LIMIT $max_results
            """
            
            se_records = session.run(
                se_query,
                drugs=drug_nodes,
                se_rels=SIDE_EFFECT_RELATIONSHIPS,
                max_results=max_results
            )
            
            for rec in se_records:
                drug = rec['drug']
                if drug in results:
                    results[drug]['side_effects'].append({
                        'side_effect': rec['side_effect'],
                        'relation': rec['relation'],
                        'text': f"{drug} may cause {rec['side_effect']}"
                    })
            
            # Print summary
            print(f"\n{'='*60}")
            print("QUERY SUMMARY")
            print(f"{'='*60}")
            for drug in drug_nodes:
                print(f"{drug}:")
                print(f"  - Interactions: {len(results[drug]['interactions'])}")
                print(f"  - Side Effects: {len(results[drug]['side_effects'])}")
            
        except Exception as e:
            print(f"ERROR in batch query: {e}")
    
    driver.close()
    return results

In [ ]:
results = query_multi_drug_interactions(list(normalized_drugs.values()))
print(json.dumps(results, indent=4))

## Format knowledge graph results for LLM

In [69]:
def format_graph_results_for_llm(graph_results):
    context = []
    
    for drug, data in graph_results.items():
        drug_context = f"\n## Drug: {drug}\n"
        
        # Format interactions
        if data['interactions']:
            drug_context += "\n### Drug-Drug Interactions:\n"
            for interaction in data['interactions']:
                drug_context += f"- Interacts with: {interaction['interacting_drug']}\n"
        else:
            drug_context += "\n### Drug-Drug Interactions: None found\n"
        
        # Format side effects
        if data['side_effects']:
            drug_context += "\n### Side Effects:\n"
            for se in data['side_effects']:
                drug_context += f"- {se['side_effect']}\n"
        else:
            drug_context += "\n### Side Effects: None found\n"
        
        context.append(drug_context)
    
    return "\n".join(context)

In [71]:
print(format_graph_results_for_llm(results))


Querying interactions for 4 drugs...
Querying side effects for 4 drugs...

QUERY SUMMARY
Compound::DB01416:
  - Interactions: 200
  - Side Effects: 193
Compound::DB00316:
  - Interactions: 0
  - Side Effects: 7
Compound::DB00471:
  - Interactions: 0
  - Side Effects: 0
Compound::DB06812:
  - Interactions: 0
  - Side Effects: 0

## Drug: Compound::DB01416

### Drug-Drug Interactions:
- Interacts with: Compound::DB07767
- Interacts with: Compound::DB09299
- Interacts with: Compound::DB04808
- Interacts with: Compound::DB14126
- Interacts with: Compound::DB08934
- Interacts with: Compound::DB01610
- Interacts with: Compound::DB13868
- Interacts with: Compound::DB03410
- Interacts with: Compound::DB08797
- Interacts with: Compound::DB13217
- Interacts with: Compound::DB11145
- Interacts with: Compound::DB09570
- Interacts with: Compound::DB09472
- Interacts with: Compound::DB06705
- Interacts with: Compound::DB14184
- Interacts with: Compound::DB01587
- Interacts with: Compound::DB09082
-

## Use LLM (with search tool) to get proper analysis of retrieved information

In [ ]:
def analyze_drugs_with_llm(user_question, graph_results, model="gemini-2.0-flash-exp"):
    """
    Main Graph RAG function: Takes graph results and generates LLM response
    
    Args:
        user_question: The user's question about the drugs
        graph_results: Dictionary from Neo4j query (from query_multi_drug_interactions)
        model: LLM model to use
    
    Returns:
        Dictionary with question, context, and answer
    """
    print(f"\n{'='*70}")
    print("GRAPH RAG ANALYSIS")
    print(f"{'='*70}\n")
    
    # Format the knowledge graph data
    print("📝 Formatting knowledge graph data...")
    context = format_graph_results_for_llm(graph_results)
    
    # Print summary
    print(f"\n📊 Retrieved data for {len(graph_results)} drugs:")
    for drug, data in graph_results.items():
        print(f"  • {drug}: {len(data['interactions'])} interactions, {len(data['side_effects'])} side effects")
    
    # Initialize LLM
    llm = ChatGoogleGenerativeAI(
        model=model,
        temperature=0
    )
    
    # Create prompt with retrieved context
    prompt = f"""You are a pharmaceutical expert analyzing drug interactions and safety.

KNOWLEDGE GRAPH DATA (Retrieved from drug database):
{context}

USER QUESTION: {user_question}

INSTRUCTIONS:
1. Base your answer ONLY on the knowledge graph data provided above
2. Clearly identify drug interactions and their implications
3. List relevant side effects
4. Assess safety concerns if multiple drugs are involved
5. If information is missing from the data, explicitly state what is not available
6. Be specific and cite the actual drug names and side effects from the data
7. Provide actionable insights and recommendations

ANSWER:"""
    
    # Generate response
    print("\n🤖 Generating analysis with LLM...\n")
    response = llm.invoke(prompt)
    
    print(f"{'='*70}")
    print("ANSWER:")
    print(f"{'='*70}\n")
    print(response.content)
    print(f"\n{'='*70}\n")
    
    return {
        'question': user_question,
        'graph_context': context,
        'answer': response.content,
        'raw_graph_data': graph_results
    }

# Configure API keys
os.environ["GOOGLE_API_KEY"] = os.environ.get("GOOGLE_API_KEY", "")
os.environ["TAVILY_API_KEY"] = os.environ.get("TAVILY_API_KEY", "")


def format_graph_results_for_llm(graph_results):
    """
    Format Neo4j query results into structured context for LLM
    
    Input: Dictionary from query_multi_drug_interactions()
    {
        'Compound::DB00945': {
            'interactions': [...],
            'side_effects': [...]
        }
    }
    
    Output: Formatted string for LLM
    """
    context = []
    
    for drug, data in graph_results.items():
        drug_context = f"\n## Drug: {drug}\n"
        
        # Format interactions
        if data['interactions']:
            drug_context += "\n### Drug-Drug Interactions:\n"
            for interaction in data['interactions']:
                drug_context += f"- Interacts with: {interaction['interacting_drug']}\n"
        else:
            drug_context += "\n### Drug-Drug Interactions: None found\n"
        
        # Format side effects
        if data['side_effects']:
            drug_context += "\n### Side Effects:\n"
            for se in data['side_effects']:
                drug_context += f"- {se['side_effect']}\n"
        else:
            drug_context += "\n### Side Effects: None found\n"
        
        context.append(drug_context)
    
    return "\n".join(context)


def analyze_drugs_with_llm(user_question, graph_results, model="gemini-2.0-flash-exp"):
    """
    Main Graph RAG function: Takes graph results and generates LLM response
    
    Args:
        user_question: The user's question about the drugs
        graph_results: Dictionary from Neo4j query (from query_multi_drug_interactions)
        model: LLM model to use
    
    Returns:
        Dictionary with question, context, and answer
    """
    print(f"\n{'='*70}")
    print("GRAPH RAG ANALYSIS")
    print(f"{'='*70}\n")
    
    # Format the knowledge graph data
    print("📝 Formatting knowledge graph data...")
    context = format_graph_results_for_llm(graph_results)
    
    # Print summary
    print(f"\n📊 Retrieved data for {len(graph_results)} drugs:")
    for drug, data in graph_results.items():
        print(f"  • {drug}: {len(data['interactions'])} interactions, {len(data['side_effects'])} side effects")
    
    # Initialize LLM
    llm = ChatGoogleGenerativeAI(
        model=model,
        temperature=0
    )
    
    # Create prompt with retrieved context
    prompt = f"""You are a pharmaceutical expert analyzing drug interactions and safety.

KNOWLEDGE GRAPH DATA (Retrieved from drug database):
{context}

USER QUESTION: {user_question}

INSTRUCTIONS:
1. Base your answer ONLY on the knowledge graph data provided above
2. Clearly identify drug interactions and their implications
3. List relevant side effects
4. Assess safety concerns if multiple drugs are involved
5. If information is missing from the data, explicitly state what is not available
6. Be specific and cite the actual drug names and side effects from the data
7. Provide actionable insights and recommendations

ANSWER:"""
    
    # Generate response
    print("\n🤖 Generating analysis with LLM...\n")
    response = llm.invoke(prompt)
    
    print(f"{'='*70}")
    print("ANSWER:")
    print(f"{'='*70}\n")
    print(response.content)
    print(f"\n{'='*70}\n")
    
    return {
        'question': user_question,
        'graph_context': context,
        'answer': response.content,
        'raw_graph_data': graph_results
    }


def analyze_drug_combination(graph_results, model="gemini-2.0-flash-exp", use_search=True):
    """
    Specialized function for analyzing drug combinations with human-readable output
    Now includes Tavily search for enhanced drug name resolution and safety info
    
    Args:
        graph_results: Dictionary from Neo4j query
        model: LLM model to use
        use_search: Whether to use Tavily search for additional context
    
    Returns:
        Safety analysis of the drug combination with proper drug names
    """
    drug_codes = list(graph_results.keys())
    
    print(f"\n{'='*70}")
    print(f"ANALYZING COMBINATION: {len(drug_codes)} drugs")
    print(f"{'='*70}\n")
    
    # Format context
    context = format_graph_results_for_llm(graph_results)
    
    # Check for direct interactions between input drugs
    mutual_interactions = []
    for drug1 in drug_codes:
        for drug2 in drug_codes:
            if drug1 != drug2:
                for interaction in graph_results[drug1]['interactions']:
                    if interaction['interacting_drug'] == drug2:
                        mutual_interactions.append(f"{drug1} ↔ {drug2}")
    
    # Initialize LLM
    llm = ChatGoogleGenerativeAI(
        model=model,
        temperature=0.1
    )
    
    if use_search:
        try:
            from tavily import TavilyClient
            
            print("🔍 Searching for drug information with Tavily...\n")
            tavily = TavilyClient(api_key=TAVILY_API_KEY)
            
            # Create Tavily search tool
            def tavily_search(query: str) -> str:
                """Search for drug information and safety data"""
                try:
                    response = tavily.search(
                        query=query,
                        search_depth="advanced",
                        max_results=5
                    )
                    results = []
                    for r in response.get("results", []):
                        results.append(f"Title: {r['title']}\nContent: {r['content']}\nSource: {r['url']}\n")
                    return "\n---\n".join(results) if results else "No additional information found"
                except Exception as e:
                    print(f"Search error: {e}")
                    return "Search unavailable"
            
            tools = [
                Tool(
                    name="drug_search",
                    func=tavily_search,
                    description="Search for drug names, interactions, and safety information. Use this to convert drug codes to names and find additional safety data."
                )
            ]
            
            # Create agent with search capability
            prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a pharmaceutical expert with access to web search.

CRITICAL INSTRUCTIONS:
1. First, use the drug_search tool to find the common names for each drug code in the heading and optionally the drug codes in interactions (as there can by many and searching each is not efficient)
2. Search for "[drug code] drug name" for each code
3. Then search for safety information about the specific drug combinations
4. Use search results to enhance your analysis with current information

After gathering information, provide comprehensive analysis in the specified format.
Use plain language and convert ALL codes to drug names."""),
                ("human", """DRUGS IN COMBINATION:
{drug_list}

KNOWLEDGE GRAPH DATA:
{context}

DIRECT INTERACTIONS DETECTED:
{interactions}

PROVIDE YOUR ANALYSIS IN THIS FORMAT:

## 1. Drug Names (use search to confirm)
List each drug code with its common name

## 2. Overall Safety Assessment
Risk level: [Low/Moderate/High/Critical]
Brief explanation (2-3 sentences)

## 3. Drug Interactions Explained
For each interaction found:
- Which drugs interact
- What happens when they interact (mechanism)
- Clinical significance

## 4. Popular Drug Interactions
Explain interactions with commonly prescribed drugs:
- Mention specific drug names
- Explain the interaction simply
- State the clinical impact

## 5. Common Side Effects
List in plain language:
- Group similar side effects
- Indicate severity
- Mention which drugs cause which effects

## 6. Risk Analysis
Brief assessment:
- Most concerning combinations
- Cumulative effects
- Special populations at risk

## 7. Clinical Recommendations
- Can these be taken together?
- Timing considerations
- Monitoring needed
- When to seek medical attention

## 8. Data Limitations
What information is missing

Use the drug_search tool as needed to enhance your analysis."""),
                ("placeholder", "{agent_scratchpad}")
            ])
            
            # Create agent
            agent = create_tool_calling_agent(llm, tools, prompt)
            agent_executor = AgentExecutor(
                agent=agent,
                tools=tools,
                verbose=True,
                max_iterations=5,
                handle_parsing_errors=True
            )
            
            # Execute with agent
            print("🤖 Generating comprehensive analysis with search...\n")
            result = agent_executor.invoke({
                "drug_list": "\n".join(f'{i+1}. {drug}' for i, drug in enumerate(drug_codes)),
                "context": context,
                "interactions": "\n".join(mutual_interactions) if mutual_interactions else "No direct interactions found in database"
            })
            
            analysis = result['output']
            
        except ImportError:
            print("⚠️  Tavily not installed. Install with: pip install tavily-python")
            print("Falling back to basic analysis...\n")
            use_search = False
        except Exception as e:
            print(f"⚠️  Search failed: {e}")
            print("Falling back to basic analysis...\n")
            use_search = False
    
    # Fallback: Basic analysis without search
    if not use_search:
        prompt = f"""You are a pharmaceutical expert. Analyze this drug combination for safety concerns.

IMPORTANT INSTRUCTIONS:
- Convert ALL drug codes (Compound::DB##### or CHEMBL#####) to their common/generic drug names
- Use simple, clear language that patients can understand
- Explain interactions in plain terms
- For side effects, use common terms
- Keep explanations brief but informative (2-3 sentences per point)

DRUGS IN COMBINATION:
{chr(10).join(f'{i+1}. {drug}' for i, drug in enumerate(drug_codes))}

KNOWLEDGE GRAPH DATA:
{context}

DIRECT INTERACTIONS DETECTED:
{chr(10).join(mutual_interactions) if mutual_interactions else "No direct interactions found in database"}

PROVIDE YOUR ANALYSIS IN THIS FORMAT:

## 1. Drug Names
List each drug code with its common name

## 2. Overall Safety Assessment
Risk level: [Low/Moderate/High/Critical]
Brief explanation (2-3 sentences)

## 3. Drug Interactions Explained
For each interaction found:
- Which drugs interact
- What happens when they interact
- Clinical significance

## 4. Popular Drug Interactions
Explain interactions with commonly prescribed drugs

## 5. Common Side Effects
List in plain language with severity indicators

## 6. Risk Analysis
Brief assessment of concerns and cumulative effects

## 7. Clinical Recommendations
- Can these be taken together?
- Timing and monitoring guidance

## 8. Data Limitations
What information is missing

ANALYSIS:"""
        
        print("🤖 Generating safety analysis...\n")
        response = llm.invoke(prompt)
        analysis = response.content
    
    print(f"{'='*70}")
    print("SAFETY ANALYSIS:")
    print(f"{'='*70}\n")
    print(analysis)
    print(f"\n{'='*70}\n")
    
    return {
        'drug_codes': drug_codes,
        'mutual_interactions': mutual_interactions,
        'analysis': analysis,
        'raw_graph_data': graph_results,
        'used_search': use_search
    }

In [ ]:
output = analyze_drug_combination(results)

## Graph-RAG pipeline (WIP)

In [ ]:
import os
from neo4j import GraphDatabase
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate
from langchain.tools import Tool
from langchain_core.messages import HumanMessage, SystemMessage

# ---------- NEO4J CONFIG ----------
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "12345678"

# ---------- RELATIONSHIP PATTERNS ----------
DDI_RELATIONSHIPS = ["DRUGBANK::ddi-interactor-in::Compound:Compound"]
SIDE_EFFECT_RELATIONSHIPS = [
    "Hetionet::CcSE::Compound:Side Effect",
    "GNBR::Sa::Compound:Disease"
]

# ---------- CONFIGURE GEMINI ----------
os.environ["GOOGLE_API_KEY"] = os.environ.get("GOOGLE_API_KEY", "")


class DrugKnowledgeGraphRAG:
    """Graph RAG system for drug interaction analysis"""
    
    def __init__(self, neo4j_uri=NEO4J_URI, neo4j_user=NEO4J_USER, neo4j_password=NEO4J_PASSWORD):
        self.driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash-exp",
            temperature=0
        )
    
    def retrieve_drug_context(self, drug_nodes, max_results=200):
        """
        Retrieve drug interactions and side effects from knowledge graph
        """
        results = {drug: {'interactions': [], 'side_effects': []} for drug in drug_nodes}
        
        with self.driver.session() as session:
            try:
                # Query drug-drug interactions
                ddi_query = """
                UNWIND $drugs AS drug_id
                MATCH (drug {Entity: drug_id})-[r]-(partner)
                WHERE r.Relationship IN $ddi_rels
                RETURN DISTINCT 
                    drug.Entity AS drug,
                    r.Relationship AS relation,
                    partner.Entity AS interacting_drug
                LIMIT $max_results
                """
                
                ddi_records = session.run(
                    ddi_query,
                    drugs=drug_nodes,
                    ddi_rels=DDI_RELATIONSHIPS,
                    max_results=max_results
                )
                
                for rec in ddi_records:
                    drug = rec['drug']
                    if drug in results:
                        results[drug]['interactions'].append({
                            'interacting_drug': rec['interacting_drug'],
                            'relation': rec['relation']
                        })
                
                # Query side effects
                se_query = """
                UNWIND $drugs AS drug_id
                MATCH (drug {Entity: drug_id})-[r]-(effect)
                WHERE r.Relationship IN $se_rels
                RETURN DISTINCT
                    drug.Entity AS drug,
                    r.Relationship AS relation,
                    effect.Entity AS side_effect
                LIMIT $max_results
                """
                
                se_records = session.run(
                    se_query,
                    drugs=drug_nodes,
                    se_rels=SIDE_EFFECT_RELATIONSHIPS,
                    max_results=max_results
                )
                
                for rec in se_records:
                    drug = rec['drug']
                    if drug in results:
                        results[drug]['side_effects'].append({
                            'side_effect': rec['side_effect'],
                            'relation': rec['relation']
                        })
                
            except Exception as e:
                print(f"Error retrieving context: {e}")
        
        return results
    
    def format_context_for_llm(self, graph_data):
        """
        Format retrieved graph data into structured context for LLM
        """
        context = []
        
        for drug, data in graph_data.items():
            drug_context = f"\n## {drug}\n"
            
            # Drug interactions
            if data['interactions']:
                drug_context += "\n### Drug Interactions:\n"
                for interaction in data['interactions']:
                    drug_context += f"- Interacts with: {interaction['interacting_drug']}\n"
            else:
                drug_context += "\n### Drug Interactions: None found in database\n"
            
            # Side effects
            if data['side_effects']:
                drug_context += "\n### Known Side Effects:\n"
                for se in data['side_effects']:
                    drug_context += f"- {se['side_effect']}\n"
            else:
                drug_context += "\n### Side Effects: None found in database\n"
            
            context.append(drug_context)
        
        return "\n".join(context)
    
    def query_with_rag(self, user_question, drug_nodes):
        """
        Main Graph RAG function: Retrieve from KG + Generate with LLM
        """
        print(f"\n{'='*70}")
        print("GRAPH RAG PIPELINE")
        print(f"{'='*70}\n")
        
        # Step 1: RETRIEVE - Get relevant data from knowledge graph
        print("📊 Step 1: Retrieving from Knowledge Graph...")
        graph_data = self.retrieve_drug_context(drug_nodes)
        
        # Print retrieval summary
        for drug, data in graph_data.items():
            print(f"  ✓ {drug}: {len(data['interactions'])} interactions, {len(data['side_effects'])} side effects")
        
        # Step 2: FORMAT - Structure the context
        print("\n📝 Step 2: Formatting context for LLM...")
        context = self.format_context_for_llm(graph_data)
        
        # Step 3: AUGMENT - Create enhanced prompt with retrieved context
        print("\n🤖 Step 3: Generating answer with LLM...")
        
        prompt = f"""You are a pharmaceutical expert analyzing drug interactions and safety.

KNOWLEDGE GRAPH DATA:
{context}

USER QUESTION: {user_question}

Instructions:
1. Analyze the knowledge graph data provided above
2. Focus on drug interactions and side effects
3. Identify potential risks or concerns
4. Provide clear, actionable insights
5. If data is missing, clearly state what information is not available
6. Be precise and cite specific interactions/side effects from the data

Provide a comprehensive analysis:"""
        
        response = self.llm.invoke(prompt)
        
        print(f"\n{'='*70}")
        print("ANSWER:")
        print(f"{'='*70}\n")
        print(response.content)
        
        return {
            'question': user_question,
            'retrieved_context': graph_data,
            'formatted_context': context,
            'answer': response.content
        }
    
    def create_agent_with_kg_tool(self, drug_nodes):
        """
        Create an agent that can query the knowledge graph as needed
        """
        # Define KG query tool
        def kg_query_tool(query: str) -> str:
            """Query the drug knowledge graph for interactions and side effects"""
            graph_data = self.retrieve_drug_context(drug_nodes)
            return self.format_context_for_llm(graph_data)
        
        tools = [
            Tool(
                name="drug_knowledge_graph",
                func=kg_query_tool,
                description="Query the drug knowledge graph to get information about drug interactions and side effects. Use this when asked about drug safety, interactions, or side effects."
            )
        ]
        
        # Create prompt
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a pharmaceutical expert with access to a drug knowledge graph.

When answering questions about drugs:
1. ALWAYS use the drug_knowledge_graph tool first to get factual data
2. Analyze the retrieved information carefully
3. Provide evidence-based answers citing specific interactions or side effects
4. Be clear about what data is available vs missing
5. Focus on safety and clinical relevance

Available drugs in context: {drugs}"""),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}")
        ])
        
        # Create agent
        agent = create_tool_calling_agent(self.llm, tools, prompt)
        agent_executor = AgentExecutor(
            agent=agent,
            tools=tools,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=3
        )
        
        return agent_executor
    
    def analyze_drug_combination(self, drug_nodes, return_graph_data=False):
        """
        Specialized function to analyze drug combinations
        """
        print(f"\n{'='*70}")
        print(f"ANALYZING DRUG COMBINATION: {', '.join(drug_nodes)}")
        print(f"{'='*70}\n")
        
        # Retrieve data
        graph_data = self.retrieve_drug_context(drug_nodes)
        
        # Find cross-interactions
        all_interacting_drugs = set()
        for drug, data in graph_data.items():
            for interaction in data['interactions']:
                all_interacting_drugs.add(interaction['interacting_drug'])
        
        # Check if any of the input drugs interact with each other
        mutual_interactions = []
        for drug1 in drug_nodes:
            for drug2 in drug_nodes:
                if drug1 != drug2:
                    for interaction in graph_data[drug1]['interactions']:
                        if interaction['interacting_drug'] == drug2:
                            mutual_interactions.append((drug1, drug2))
        
        # Format context
        context = self.format_context_for_llm(graph_data)
        
        # Generate analysis
        prompt = f"""Analyze this drug combination for safety concerns.

DRUGS BEING COMBINED:
{', '.join(drug_nodes)}

KNOWLEDGE GRAPH DATA:
{context}

DETECTED MUTUAL INTERACTIONS:
{mutual_interactions if mutual_interactions else "None detected in database"}

Provide:
1. **Safety Assessment**: Overall risk level (Low/Moderate/High)
2. **Direct Interactions**: Do these drugs interact with each other?
3. **Cumulative Side Effects**: Overlapping or compounding side effects
4. **Recommendations**: Clinical guidance for safe use
5. **Missing Data**: What information is not available

Analysis:"""
        
        response = self.llm.invoke(prompt)
        
        result = {
            'drugs': drug_nodes,
            'mutual_interactions': mutual_interactions,
            'graph_data': graph_data if return_graph_data else None,
            'analysis': response.content
        }
        
        print(response.content)
        return result
    
    def close(self):
        """Close Neo4j connection"""
        self.driver.close()


# ---------- EXAMPLE USAGE ----------
if __name__ == "__main__":
    # Initialize Graph RAG system
    rag = DrugKnowledgeGraphRAG()
    
    # Example drugs (replace with your normalized drug IDs)
    test_drugs = [
        "Compound::DB00945",  # Aspirin
        "Compound::DB01050",  # Ibuprofen
    ]
    
    print("\n" + "="*70)
    print("EXAMPLE 1: Simple RAG Query")
    print("="*70)
    
    # Simple RAG query
    result1 = rag.query_with_rag(
        "What are the interactions and side effects of these drugs?",
        test_drugs
    )
    
    print("\n" + "="*70)
    print("EXAMPLE 2: Drug Combination Analysis")
    print("="*70)
    
    # Analyze drug combination
    result2 = rag.analyze_drug_combination(test_drugs)
    
    print("\n" + "="*70)
    print("EXAMPLE 3: Agent-based Query")
    print("="*70)
    
    # Create agent
    agent = rag.create_agent_with_kg_tool(test_drugs)
    
    # Query with agent
    agent_result = agent.invoke({
        "input": "Can these drugs be taken together safely?",
        "drugs": ", ".join(test_drugs)
    })
    
    print("\n" + "="*70)
    print("Agent Answer:")
    print("="*70)
    print(agent_result['output'])
    
    # Close connection
    rag.close()